## GED conflict data clean up

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
##set a working directory
user = os.path.expanduser('~')
display(user)

os.makedirs(f'{user}/Desktop/thesis/', exist_ok=True)
path = os.chdir(f'{user}/Desktop/'+'thesis/')

'/Users/NatStrom'

In [3]:
data = pd.read_csv('GEDEvent_v22_1.csv')

/var/folders/3l/pgxm2f9j3bbgbjcj2pyxl__00000gp/T/ipykernel_35756/2663555733.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('GEDEvent_v22_1.csv')


In [4]:
df = data.copy()

In [5]:
print(df.dtypes)
print(df.columns)
print(df.describe(include='all'))

## sanity check to ensure the spread of the data is correct
print('sanity check 1, ensure appropriate years and countries are represented')
list_country=list(list(df['country_id'].drop_duplicates()))
list_year=list(list(df['year'].drop_duplicates()))
print('country list', list_country)
print('year list', list_year)
#inspect for NAs
print('following checks for completeness of all columns')
print(df['country_id'].isnull().values.any())

id                     int64
relid                 object
year                   int64
active_year            int64
code_status           object
type_of_violence       int64
conflict_dset_id       int64
conflict_new_id        int64
conflict_name         object
dyad_dset_id           int64
dyad_new_id            int64
dyad_name             object
side_a_dset_id         int64
side_a_new_id          int64
side_a                object
side_b_dset_id         int64
side_b_new_id          int64
side_b                object
number_of_sources      int64
source_article        object
source_office         object
source_date           object
source_headline       object
source_original       object
where_prec             int64
where_coordinates     object
where_description     object
adm_1                 object
adm_2                 object
latitude             float64
longitude            float64
geom_wkt              object
priogrid_gid           int64
country               object
country_id    

In [6]:
#filter only the wanted columns
df_filtered = df.filter(['id', 'relid', 'year', 'type_of_violence','conflict_new_id','country_id','best', 'high', 'low'])

In [7]:
display(df_filtered)

,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low
0,244657,IRQ-2017-1-524-322,2017,1,259,700,6,6,6
1,412700,IRQ-2021-1-524-145,2021,1,259,700,183,184,171
2,413023,IRQ-2021-1-524-143,2021,1,259,700,2,3,0
3,412909,IRQ-2021-1-524-144,2021,1,259,700,10,10,9
4,132140,AFG-1989-1-411-2,1989,1,333,700,6,6,6
...,...,...,...,...,...,...,...,...,...
293629,15525,MZM-1989-3-1347-4,1989,3,562,552,9,9,9
293630,15524,MZM-1989-3-1347-16,1989,3,562,552,9,9,9
293631,15245,MZM-1990-3-1347-18,1990,3,562,552,7,7,7
293632,15516,MZM-1990-3-1347-28,1990,3,562,552,1,1,1


In [8]:
from ingester3.scratch import cache_manager

cache_manager()

# This will import the Pandas extensions as well as bootstrap the object layer
from ingester3.extensions import *

In [9]:
#create the c_id from the gw codes
#create c_ids
df_filtered = pd.DataFrame.cy.soft_validate_gwcode_year(df_filtered, gw_col='country_id', year_col = 'year', at_month = 12)
inspect=df_filtered.query("valid_id == False")
print('countries with invalid id below')
display(inspect)

df_filtered = pd.DataFrame.c.from_gwcode(df_filtered, gw_col='country_id', month_col=None)

countries with invalid id below


,id,relid,year,type_of_violence,conflict_new_id,country_id,best,high,low,valid_id
138842,16107,SAF-1989-3-560-14,1989,3,477,565,0,300,0,False
138843,17748,SAF-1989-3-560-12,1989,3,477,565,0,27,0,False


In [10]:
df_filtered['Africa'] = df_filtered.c.in_africa

DatabaseError: (psycopg2.DatabaseError) could not receive data from server: Operation timed out
SSL SYSCALL error: Operation timed out

[SQL: 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (n.nspname = %(schema)s)
            AND c.relname = %(table_name)s AND c.relkind in
            ('r', 'v', 'm', 'f', 'p')
        ]
[parameters: {'schema': 'prod_metadata', 'table_name': 'update_stamp'}]
(Background on this error at: https://sqlalche.me/e/14/4xp6)

In [ ]:
df_filtered = df_filtered.query('Africa == True')

In [ ]:
#get the iso numbers
df_filtered['iso'] = df_filtered.c.isoab

In [ ]:
inspect = df_filtered.loc[df_filtered.iso.isna()]
display(inspect)

In [ ]:
df_conflict = df_filtered.copy()

In [ ]:
df_conflict = df_conflict.filter(['iso','year','type_of_violence', 'best', 'high', 'low'])
df_conflict

In [ ]:
#now produce the lags and filter out the unwanted years
#now I need to loop the lagging function, creating a new column and appending them all together to then have a dataframe i can concanate

df_conflict['lag_best_5'] = conflict.groupby('iso')['best'].shift(5)
df_conflict['lag_best_10'] = conflict.groupby('iso')['best'].shift(10)
df_conflict['lag_best_20'] = conflict.groupby('iso')['best'].shift(20)
    

df_conflict = df_conflict.loc[df_conflict['year'] >= 1990]
df_conflict

In [ ]:
#replace row values
df_conflict['type_of_violence'] = df_conflict['type_of_violence'].map({1:'sb',
                                            2:'ns',
                                            3:'os'})

In [ ]:
#create dummies
def categorize(row):  
    if row['type_of_violence'] == 'sb':
        return 1
    return 0

df_conflict['dummy_sb'] = df_conflict.apply(lambda row: categorize(row), axis=1)

# repeat for non-state violence
def categorize(row):  
    if row['type_of_violence'] == 'ns':
        return 1
    return 0

df_conflict['dummy_ns'] = df_conflict.apply(lambda row: categorize(row), axis=1)

#repeat dummy creation for one-sided violence
def categorize(row):  
    if row['type_of_violence'] == 'os':
        return 1
    return 0

df_conflict['dummy_os'] = df_conflict.apply(lambda row: categorize(row), axis=1)



In [ ]:
#add dummies for the type of violence
df_conflict

In [ ]:
#create dummies for intensity
def categorize(row):  
    if row['best'] <= 100:
        return 'low'
    if row['best'] > 100 or row['best'] < 1000:
        return 'medium'
    return 'high'

df_conflict['intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)
df_conflict

In [ ]:
# create dummy for low intensity conflict
def categorize(row):  
    if row['intensity'] == 'low':
        return 1
    return 0

df_conflict['dummy_low_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

#repeat dummy creation for medium intensity
def categorize(row):  
    if row['intensity'] == 'middle':
        return 1
    return 0

df_conflict['dummy_medium_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

def categorize(row):  
    if row['intensity'] == 'high':
        return 1
    return 0

df_conflict['dummy_high_intensity'] = df_conflict.apply(lambda row: categorize(row), axis=1)

In [ ]:
df_conflict

In [ ]:
#create the neighbouring country at war index, best >= 25
#first try it for one country
dza_neighbors = Country.from_iso('DZA', month_id=455).neighbors()

print('dza neighbour ID: ', dza_neighbors)
print()
print('dza neighbour names:', [i.name for i in dza_neighbors])

In [ ]:
list_c = list(df_conflict.iso.unique())

In [ ]:
neighbors = list()
for i in list_c:
    neighbors = Country.from_iso(i,month_id =455).neighbors()
    neighbors_all = neighbors.append()

In [ ]:
neighbors

In [ ]:
#create columns for all scenarios?
#adding the different dummies for intensity, type and exposure
data_sb = data.query('dummy_sb == 1')
data_os = data.query('dummy_os == 1')
data_ns =data.query('dummy_ns == 1') 

data_low_intensity = data.query('dummy_low_intensity == 1')
data_medium_intensity = data.query('dummy_medium_intensity == 1')
data_high_intensity = data.query('dummy_high_intensity == 1')v

In [ ]:
df_pivot = df_filtered.pivot_table(index=['iso','year_id'],columns=['sex_id','age_group'],values=['best'],  aggfunc= 'sum')
##index = the columns you want to keep as index of the df
##columns= the columns you want pivoted as individual columns
##values= the column that conttains the individual value for the newly pivoted columns

#now unstack the dataframe to get the wished result
df_pivot.assign(**df_pivot.index.to_frame()) #need to do this if the data is multihierarchical
df_pivot = df_pivot.droplevel(level=0,axis=1) #if the pivoting results in a multi-levels, you need to drop the respective level
df_pivot.columns = ['_'.join(col) for col in dta_diss.columns]#join together all the column names
df_pivot= df_pivot.reset_index()
print(df_pivot)

In [ ]:
df_conflict.to_csv('df_conflict.csv')